In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv("../../data/signate-bankcustomer/train.csv")
test = pd.read_csv("../../data/signate-bankcustomer/test.csv")
sample_sub = pd.read_csv("../../data/signate-bankcustomer/submit_sample.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27100 entries, 0 to 27099
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         27100 non-null  int64 
 1   age        27100 non-null  int64 
 2   job        27100 non-null  object
 3   marital    27100 non-null  object
 4   education  27100 non-null  object
 5   default    27100 non-null  object
 6   balance    27100 non-null  int64 
 7   housing    27100 non-null  object
 8   loan       27100 non-null  object
 9   contact    27100 non-null  object
 10  day        27100 non-null  int64 
 11  month      27100 non-null  object
 12  duration   27100 non-null  int64 
 13  campaign   27100 non-null  int64 
 14  pdays      27100 non-null  int64 
 15  previous   27100 non-null  int64 
 16  poutcome   27100 non-null  object
 17  y          27100 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.7+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18050 entries, 0 to 18049
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         18050 non-null  int64 
 1   age        18050 non-null  int64 
 2   job        18050 non-null  object
 3   marital    18050 non-null  object
 4   education  18050 non-null  object
 5   default    18050 non-null  object
 6   balance    18050 non-null  int64 
 7   housing    18050 non-null  object
 8   loan       18050 non-null  object
 9   contact    18050 non-null  object
 10  day        18050 non-null  int64 
 11  month      18050 non-null  object
 12  duration   18050 non-null  int64 
 13  campaign   18050 non-null  int64 
 14  pdays      18050 non-null  int64 
 15  previous   18050 non-null  int64 
 16  poutcome   18050 non-null  object
dtypes: int64(8), object(9)
memory usage: 2.3+ MB


In [6]:
train_test = pd.concat([train,test],axis=0)

In [7]:
train_test["y"] = train_test["y"].fillna(-999)

## 1.age

In [8]:
train_test["age"] = np.clip(train_test["age"], train["age"].min(), train["age"].max())

In [9]:
train_test["age"] = np.log(train_test["age"])
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 2.job

In [10]:
test.loc[test["job"]=="unknown","job"] = "blue-collar"
train_test = pd.concat([train,test],axis=0)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
#ラベルエンコーディングよりカウントエンコーディングの方が精度が良かったので変更
#columns = train_test["job"].unique()
#le = LabelEncoder()
#encoded = le.fit_transform(train_test["job"].values)
#decoded = le.inverse_transform(encoded)
#train_test['job'] = pd.to_numeric(encoded)

In [12]:
#pd.DataFrame(
#    {'変換前': columns,
#     '変換後': le.transform(columns)})

In [13]:
train_test["job"] = train_test["job"].map(train_test["job"].value_counts())

In [14]:
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 3.marital

In [15]:
train_test["marital"] = train_test["marital"].map(pd.Series({"single":0,"divorced":1,"married":2}))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 4.education

In [16]:
#欠損値にも意味がありそうだったので欠損値フラグを作成
train_test["edu_unknown_flg"] = train_test["education"].apply(lambda x:1 if x=="unknown" else 0)

In [17]:
#label encoding
#train_test["education"] = train_test["education"].map(pd.Series({"unknown":0,"primary":1,"secondary":2,"tertiary":3}))

#freq encoding
train_test["education"] = train_test["education"].map(train_test["education"].value_counts())

train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 5.default

In [18]:
train["default"].value_counts()

no     27090
yes       10
Name: default, dtype: int64

In [19]:
test["default"].value_counts()

no     18046
yes        4
Name: default, dtype: int64

In [20]:
columns = train_test["default"].unique()
le = LabelEncoder()
encoded = le.fit_transform(train_test["default"].values)
decoded = le.inverse_transform(encoded)
train_test['default'] = pd.to_numeric(encoded)
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [21]:
pd.DataFrame(
    {'変換前': columns,
     '変換後': le.transform(columns)})

,変換前,変換後
0,no,0
1,yes,1


## 6.balance

In [22]:
#収入がマイナスかどうかを表すフラグを作成
def set_balance_flg(x):
    if x < 0:
        return 1
    else:
        return 0

In [23]:
train_test["balance_minus_flg"] = train_test["balance"].apply(lambda x:set_balance_flg(x))
train_test["balance"] = np.clip(train_test["balance"], train["balance"].min(), train["balance"].max())
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 7.housing

In [24]:
columns = train_test["housing"].unique()
le = LabelEncoder()
encoded = le.fit_transform(train_test["housing"].values)
decoded = le.inverse_transform(encoded)
train_test['housing'] = pd.to_numeric(encoded)
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [25]:
pd.DataFrame(
    {'変換前': columns,
     '変換後': le.transform(columns)})

,変換前,変換後
0,yes,1
1,no,0


## 8.loan


In [26]:
columns = train_test["loan"].unique()
le = LabelEncoder()
encoded = le.fit_transform(train_test["loan"].values)
decoded = le.inverse_transform(encoded)
train_test['loan'] = pd.to_numeric(encoded)
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [27]:
pd.DataFrame(
    {'変換前': columns,
     '変換後': le.transform(columns)})

,変換前,変換後
0,no,0
1,yes,1


## 9.contact

In [28]:
train_test["con_unknown_flg"] = train_test["contact"].apply(lambda x:1 if x=="unknown" else 0)

In [29]:
train_test["contact"] = train_test["contact"].map(pd.Series({"unknown":0,"cellular":1,"telephone":2}))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 10.day

In [30]:
#あとでまとめて標準化するので変更なし

## 11.month

In [31]:
train["month"].unique()

array(['nov', 'aug', 'may', 'apr', 'sep', 'jun', 'jul', 'feb', 'oct',
       'jan', 'mar'], dtype=object)

In [32]:
test["month"].unique()

array(['feb', 'jul', 'may', 'jan', 'jun', 'aug', 'apr', 'sep', 'nov',
       'oct', 'mar'], dtype=object)

In [33]:
#月を表す文字列を数値に変更
def get_intmonth(x):
    if x == "jan":
        x = 1
    elif x == "feb":
        x = 2
    elif x == "mar":
        x = 3
    elif x == "apr":
        x = 4
    elif x == "may":
        x = 5
    elif x == "jun":
        x = 6
    elif x == "jul":
        x = 7
    elif x == "aug":
        x = 8
    elif x == "sep":
        x = 9
    elif x == "oct":
        x = 10
    elif x == "nov":
        x = 11
    elif x == "dec":
        x = 12
    
    return x

In [34]:
train_test["month"] = train_test["month"].apply(lambda x:get_intmonth(x))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [35]:
train_test["month+day"] = train_test["month"].astype(str) + train_test["day"].astype(str)
train_test["month+day"] = train_test["month+day"].astype(int)
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 12.duration

In [36]:
#連絡をとった時間が０秒かどうかのフラグを作成
def set_duration_flg(x):
    if x == 0:
        return 1
    else:
        return 0

In [37]:
train_test["duration_zero_flg"] = train_test["duration"].apply(lambda x:set_duration_flg(x))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 13.campaign

In [38]:
#campaignが５のデータは１つしかないため、値を４に変更する
train_test.loc[train_test["campaign"]==5,"campaign"] = 4

In [39]:
#label encoding
train_test["campaign"] = train_test["campaign"].map(pd.Series({1:4,2:3,3:2,4:1}))

#ラベルエンコーディングの方が精度が良かったので変更
#freq encoding
#train_test["campaign"] = train_test["campaign"].map(train_test["campaign"].value_counts())

train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 14.pdays

In [40]:
#最後に連絡をとった日からの経過日数がマイナス(おそらく欠損値)を表すフラグを作成
def set_pdays_flg(x):
    if x == -1:
        return 0
    else:
        return 1

In [41]:
train_test["pdays_flg"] = train_test["pdays"].apply(lambda x:set_pdays_flg(x))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## 15.previous

In [42]:
#変更なし

## 16.poutcome

In [43]:
#欠損値にも意味があると考えたため、欠損値であることを表すフラグを作成
train_test["pou_unknown_flg"] = train_test["poutcome"].apply(lambda x:1 if x=="unknown" else 0)

In [44]:
train_test["poutcome"] = train_test["poutcome"].map(pd.Series({"success":0,"failure":1,"other":2,"unknown":3}))
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

## ダミー変数化、標準化

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
sc = StandardScaler()

In [47]:
#duration,pdaysは欠損値予測をするため別で処理する
num_columns = ["age","balance","day","month+day"]

In [48]:
sc.fit(train_test[num_columns])
df_num = pd.DataFrame(sc.transform(train_test[num_columns]),columns = num_columns)

In [49]:
#sns.distplot((df_num["education"]))

In [50]:
train_test = train_test.drop(num_columns,axis=1)
train_test = pd.concat([train_test.reset_index(drop=True),df_num],axis=1)

In [51]:
train_test.loc[train_test["duration"]==0,"duration"] = -999
train_test.loc[train_test["pdays"]==-1,"pdays"] = -999

In [52]:
#duration,pdaysは欠損値があるので、欠損値を除いたデータを標準化
duration_index = train_test.loc[train_test["duration"]!=-999,"duration"].index.values
pdays_index = train_test.loc[train_test["pdays"]!=-999,"pdays"].index.values

In [53]:
sc = StandardScaler()
train_test.loc[duration_index,"duration"]
sc.fit(pd.DataFrame(train_test.loc[duration_index,"duration"]))
tmp_duration = pd.DataFrame(sc.transform(pd.DataFrame(train_test.loc[duration_index,"duration"]))
                            ,index=duration_index,columns=["duration_sc"])
train_test = train_test.drop("duration",axis=1)
train_test = pd.concat([train_test,tmp_duration],axis=1)

In [54]:
sc = StandardScaler()
train_test.loc[pdays_index,"pdays"]
sc.fit(pd.DataFrame(train_test.loc[pdays_index,"pdays"]))
tmp_pdays = pd.DataFrame(sc.transform(pd.DataFrame(train_test.loc[pdays_index,"pdays"]))
                            ,index=pdays_index,columns=["pdays_sc"])
train_test = train_test.drop("pdays",axis=1)
train_test = pd.concat([train_test,tmp_pdays],axis=1)

In [55]:
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [56]:
test = test.drop("y",axis=1)

In [57]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27100 entries, 0 to 27099
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 27100 non-null  int64  
 1   job                27100 non-null  int64  
 2   marital            27100 non-null  int64  
 3   education          27100 non-null  int64  
 4   default            27100 non-null  int64  
 5   housing            27100 non-null  int64  
 6   loan               27100 non-null  int64  
 7   contact            27100 non-null  int64  
 8   month              27100 non-null  int64  
 9   campaign           27100 non-null  int64  
 10  previous           27100 non-null  int64  
 11  poutcome           27100 non-null  int64  
 12  y                  27100 non-null  float64
 13  edu_unknown_flg    27100 non-null  int64  
 14  balance_minus_flg  27100 non-null  int64  
 15  con_unknown_flg    27100 non-null  int64  
 16  duration_zero_flg  271

In [58]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18050 entries, 27100 to 45149
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 18050 non-null  int64  
 1   job                18050 non-null  int64  
 2   marital            18050 non-null  int64  
 3   education          18050 non-null  int64  
 4   default            18050 non-null  int64  
 5   housing            18050 non-null  int64  
 6   loan               18050 non-null  int64  
 7   contact            18050 non-null  int64  
 8   month              18050 non-null  int64  
 9   campaign           18050 non-null  int64  
 10  previous           18050 non-null  int64  
 11  poutcome           18050 non-null  int64  
 12  edu_unknown_flg    18050 non-null  int64  
 13  balance_minus_flg  18050 non-null  int64  
 14  con_unknown_flg    18050 non-null  int64  
 15  duration_zero_flg  18050 non-null  int64  
 16  pdays_flg         

In [59]:
#決定木用のデータ
train.to_csv("../../data/signate-bankcustomer/train_fixed.csv",index=False)
test.to_csv("../../data/signate-bankcustomer/test_fixed.csv",index=False)

In [60]:
#線形モデル用にカテゴリ変数をダミー変数にしたデータを作成
#poutcomeは欠損値予測をするため別で処理する
cat_columns = ["job","marital","contact","campaign","previous","month","education"]

In [61]:
for column in cat_columns:
    tmp = pd.get_dummies(train_test[column].astype(str),drop_first=True,prefix=column)
    train_test = train_test.drop(column,axis=1)
    train_test = pd.concat([train_test,tmp],axis=1)


In [62]:
train = train_test.loc[train_test["y"]!=-999]
test = train_test.loc[train_test["y"]==-999]

In [63]:
test = test.drop("y",axis=1)

In [64]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27100 entries, 0 to 27099
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 27100 non-null  int64  
 1   default            27100 non-null  int64  
 2   housing            27100 non-null  int64  
 3   loan               27100 non-null  int64  
 4   poutcome           27100 non-null  int64  
 5   y                  27100 non-null  float64
 6   edu_unknown_flg    27100 non-null  int64  
 7   balance_minus_flg  27100 non-null  int64  
 8   con_unknown_flg    27100 non-null  int64  
 9   duration_zero_flg  27100 non-null  int64  
 10  pdays_flg          27100 non-null  int64  
 11  pou_unknown_flg    27100 non-null  int64  
 12  age                27100 non-null  float64
 13  balance            27100 non-null  float64
 14  day                27100 non-null  float64
 15  month+day          27100 non-null  float64
 16  duration_sc        270

In [65]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18050 entries, 27100 to 45149
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 18050 non-null  int64  
 1   default            18050 non-null  int64  
 2   housing            18050 non-null  int64  
 3   loan               18050 non-null  int64  
 4   poutcome           18050 non-null  int64  
 5   edu_unknown_flg    18050 non-null  int64  
 6   balance_minus_flg  18050 non-null  int64  
 7   con_unknown_flg    18050 non-null  int64  
 8   duration_zero_flg  18050 non-null  int64  
 9   pdays_flg          18050 non-null  int64  
 10  pou_unknown_flg    18050 non-null  int64  
 11  age                18050 non-null  float64
 12  balance            18050 non-null  float64
 13  day                18050 non-null  float64
 14  month+day          18050 non-null  float64
 15  duration_sc        18041 non-null  float64
 16  pdays_sc          

In [66]:
train.to_csv("../../data/signate-bankcustomer/train_fixed_dummies.csv",index=False)
test.to_csv("../../data/signate-bankcustomer/test_fixed_dummies.csv",index=False)